# 02. Initial Data Exploration, Transformation and Model Preparation

In [1]:
import gc
import os
from os import system, environ
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

                                
from ds_abm.utils.data_prep import convert_datatypes, feature_engineering, data_cleaning, create_integer_indices

In [2]:
# get the value of the PATH environment variable
DATAPATH = environ['DATAPATH']

## Data Load

In [3]:
system(f"ls -l {DATAPATH}")

total 351040
-rw-rw-r-- 1 hcekne hcekne  3941858 Sep 11 19:56 calibration_data.feather
-rw-rw-r-- 1 hcekne hcekne  1164250 Sep  5 13:36 customer_embeddings.csv
-rw-rw-r-- 1 hcekne hcekne  1439601 Sep  5 13:45 customer_embeddings_full.csv
-rw-rw-r-- 1 hcekne hcekne    25381 Aug 23 12:40 daily_new_customers.csv
-rw-rw-r-- 1 hcekne hcekne   198040 Aug 25 16:41 data_customer_grouped.csv
-rw-rw-r-- 1 hcekne hcekne  2747789 Aug 25 09:12 dataForCustomerDuration.csv
-rw-rw-r-- 1 hcekne hcekne 31902186 Sep 11 19:56 fullDataSet.feather
-rw-rw-r-- 1 hcekne hcekne 22000096 Aug 31 12:51 new_training_data_torch_prep.csv
-rw------- 1 hcekne hcekne 45622278 May 22 14:20 online_retail_II.xlsx
-rw-r--r-- 1 hcekne hcekne 45622418 Aug 17 09:23 online+retail+ii.zip
-rw-rw-r-- 1 hcekne hcekne   972926 Sep  5 13:36 product_embeddings.csv
-rw-rw-r-- 1 hcekne hcekne  1021140 Sep  5 13:49 product_embeddings_full.csv
-rw-rw-r-- 1 hcekne hcekne 83633721 Aug 31 17:01 training_data.csv
-rw-rw-r-- 1 hcekne hcekne 25

0

In [4]:
## the data contains multiple sheets and we want to load all of them:

In [5]:
# Load all sheets from the Excel file
all_sheets = pd.read_excel(DATAPATH+"online_retail_II.xlsx", sheet_name=None)

In [6]:
# Access data from each sheet
sheet_names = list(all_sheets.keys())

data_sheet1 = all_sheets[sheet_names[0]]  # This will give you the data from the first tab
data_sheet2 = all_sheets[sheet_names[1]]  # This will give you the data from the second tab

#  concatenate the two datasets into one
data = pd.concat([data_sheet1, data_sheet2], ignore_index=True)

del all_sheets, data_sheet1, data_sheet2

gc.collect()

1067522

## Preliminary Examination

In [7]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [8]:
# Summary statistics
print(data.describe())

           Quantity                    InvoiceDate         Price  \
count  1.067371e+06                        1067371  1.067371e+06   
mean   9.938898e+00  2011-01-02 21:13:55.394028544  4.649388e+00   
min   -8.099500e+04            2009-12-01 07:45:00 -5.359436e+04   
25%    1.000000e+00            2010-07-09 09:46:00  1.250000e+00   
50%    3.000000e+00            2010-12-07 15:28:00  2.100000e+00   
75%    1.000000e+01            2011-07-22 10:23:00  4.150000e+00   
max    8.099500e+04            2011-12-09 12:50:00  3.897000e+04   
std    1.727058e+02                            NaN  1.235531e+02   

         Customer ID  
count  824364.000000  
mean    15324.638504  
min     12346.000000  
25%     13975.000000  
50%     15255.000000  
75%     16797.000000  
max     18287.000000  
std      1697.464450  


In [9]:
# Info on data types and null values
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB
None


## Data Transformation

In [10]:
### To make the dateset easier to work with and have a smaller memory footprint we convert some of the datatypes into more efficient structures 

In [11]:
columns_datatypes = {
    'Invoice': 'str',
    'StockCode': 'str',
    'Description': 'str',
    'Country': 'str',
    'Quantity': 'int16',
    'Customer ID': 'float32',
    'Price': 'float32'
}

data = convert_datatypes(data, columns_datatypes=columns_datatypes)

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   Invoice          1067371 non-null  object        
 1   StockCode        1067371 non-null  object        
 2   Description      1067371 non-null  object        
 3   Quantity         1067371 non-null  int16         
 4   InvoiceDatetime  1067371 non-null  datetime64[ns]
 5   Price            1067371 non-null  float32       
 6   Customer ID      824364 non-null   float32       
 7   Country          1067371 non-null  object        
dtypes: datetime64[ns](1), float32(2), int16(1), object(4)
memory usage: 50.9+ MB


### Data Cleaning

In [13]:
# Check for missing values
print(data.isnull().sum())

Invoice                 0
StockCode               0
Description             0
Quantity                0
InvoiceDatetime         0
Price                   0
Customer ID        243007
Country                 0
dtype: int64


In [14]:
data = data_cleaning(data)

### Feature Engineering

In [15]:
data = feature_engineering(data)

In [16]:
data.head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDatetime,Price,Customer ID,Country,TotalSpending,PriceCategory,YearMonth,InvoiceDate,Year,Month,Weekday
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.399994,Very Low,2009-12,2009-12-01,2009,12,Tuesday
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.000000,Very Low,2009-12,2009-12-01,2009,12,Tuesday
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.000000,Very Low,2009-12,2009-12-01,2009,12,Tuesday
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.799995,Very Low,2009-12,2009-12-01,2009,12,Tuesday
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.000000,Very Low,2009-12,2009-12-01,2009,12,Tuesday
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085.0,United Kingdom,39.599998,Very Low,2009-12,2009-12-01,2009,12,Tuesday
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.000000,Very Low,2009-12,2009-12-01,2009,12,Tuesday
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085.0,United Kingdom,59.500000,Very Low,2009-12,2009-12-01,2009,12,Tuesday
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085.0,United Kingdom,30.599998,Very Low,2009-12,2009-12-01,2009,12,Tuesday
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085.0,United Kingdom,45.000000,Very Low,2009-12,2009-12-01,2009,12,Tuesday


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824316 entries, 0 to 824315
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   Invoice          824316 non-null  object        
 1   StockCode        824316 non-null  object        
 2   Description      824316 non-null  object        
 3   Quantity         824316 non-null  int16         
 4   InvoiceDatetime  824316 non-null  datetime64[ns]
 5   Price            824316 non-null  float32       
 6   Customer ID      824316 non-null  float32       
 7   Country          824316 non-null  object        
 8   TotalSpending    824316 non-null  float32       
 9   PriceCategory    824134 non-null  category      
 10  YearMonth        824316 non-null  period[M]     
 11  InvoiceDate      824316 non-null  period[D]     
 12  Year             824316 non-null  int32         
 13  Month            824316 non-null  int32         
 14  Weekday          824

At this point we have done most of the feature engineering and data cleaning we need to do, so we can explore the data further to enable our understanding of it and make it easier to make modelling decisions.

---

### Training, Calibration and Validation Split

In [18]:
validation_length = 2
calibration_length = 3

# Extract the most recent date in the dataset
latest_date = data['InvoiceDate'].max()

# Calculate the cutoff date for our validation set
cutoff_date_validation = pd.Period(latest_date.to_timestamp() - pd.DateOffset(months=validation_length), freq='D')
cutoff_date_calibration = pd.Period(latest_date.to_timestamp() - pd.DateOffset(months=(validation_length + calibration_length)), freq='D')

# Split the data into training, calibration and validation sets based on the cutoff dates
training_data = data[data['InvoiceDate']< cutoff_date_calibration]

calibration_data = data[(data['InvoiceDate'] >= cutoff_date_calibration) & (data['InvoiceDate'] < cutoff_date_validation)]
validation_data = data[data['InvoiceDate'] >= cutoff_date_validation]

training_data.shape, calibration_data.shape ,validation_data.shape

((600989, 15), (101756, 15), (121571, 15))

In [19]:
print(training_data['InvoiceDate'].min())
print(training_data['InvoiceDate'].max())
print(calibration_data['InvoiceDate'].min())
print(calibration_data['InvoiceDate'].max())
print(validation_data['InvoiceDate'].min())
print(validation_data['InvoiceDate'].max())

2009-12-01
2011-07-08
2011-07-10
2011-10-07
2011-10-09
2011-12-09


In [20]:
data = data.reset_index()

In [21]:
data = data.drop(columns=['index'],axis=1)

### Create indices for key training data variables

In [22]:
integer_col_dict = {'Customer ID':'CustomerIndex', 'StockCode':'ProductIndex'}

training_data = create_integer_indices(df=training_data, column_dict = integer_col_dict)


### Save the data for next steps

In [23]:
training_data.reset_index(drop=True, inplace=True)
calibration_data.reset_index(drop=True, inplace=True)
validation_data.reset_index(drop=True, inplace=True)

In [24]:
data.to_feather(DATAPATH+'fullDataSet.feather')
training_data.to_feather(DATAPATH+'training_data.feather')
calibration_data.to_feather(DATAPATH+'calibration_data.feather')
validation_data.to_feather(DATAPATH+'validation_data.feather')